In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION) # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections


def draw_styled_landmarks(image, results):
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION, 
                             mp_drawing.DrawingSpec(color=(80,110,10), thickness=1, circle_radius=1), 
                             mp_drawing.DrawingSpec(color=(80,256,121), thickness=1, circle_radius=1)
                             ) 
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 




In [3]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)

        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

f:\P\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

In [4]:
pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [5]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

In [60]:
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['Amar', 'Sonar', 'Bangla', 'Ami', 'Valobashi'])

# Thirty videos worth of data
no_sequences = 10

# Videos are going to be 30 frames in length
sequence_length = 30

In [61]:
label_map = {label:num for num, label in enumerate(actions)}

In [62]:
label_map

{'Amar': 0, 'Sonar': 1, 'Bangla': 2, 'Ami': 3, 'Valobashi': 4}

In [63]:
# Reshape the input data to fit the CNN model. 
# Here we assume that keypoints are reshaped into a 3D format for each frame in the sequence.
# The input shape would be (sequence_length, keypoint_count, 1), where '1' is for single-channel.
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

# Convert sequences into numpy arrays and reshape for CNN input
X = np.array(sequences)

In [64]:


from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

X = X.reshape((X.shape[0], 30, 1662, 1))  # shape = (num_sequences, sequence_length, keypoints, 1)
y = to_categorical(labels).astype(int)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.02)

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Building the CNN model
model = Sequential()

# First Convolutional Block
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(30, 1662, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second Convolutional Block
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Third Convolutional Block
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Flatten the output and pass it through fully connected layers
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))  # Number of actions is the output size

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


f:\P\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [66]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)               │ (None, 28, 1660, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 14, 830, 32)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 12, 828, 64)    │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 6, 414, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_8 (Conv2D)               │ (None, 4, 412, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 2, 206, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 52736)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 128)            │     6,750,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,851,589 (26.14 MB)

 Trainable params: 6,851,589 (26.14 MB)

 Non-trainable params: 0 (0.00 B)

In [67]:
from tensorflow.keras.callbacks import TensorBoard
log_dir = os.path.join('Logs_CNN_B_F_10D')
tb_callback = TensorBoard(log_dir=log_dir)

In [69]:
# Train the model
model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test), callbacks=[tb_callback])


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - categorical_accuracy: 0.8767 - loss: 0.1523 - val_categorical_accuracy: 1.0000 - val_loss: 0.1090
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 974ms/step - categorical_accuracy: 1.0000 - loss: 0.1265 - val_categorical_accuracy: 1.0000 - val_loss: 0.0718
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 939ms/step - categorical_accuracy: 0.8663 - loss: 0.1811 - val_categorical_accuracy: 1.0000 - val_loss: 0.0023
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 1s/step - categorical_accuracy: 0.9592 - loss: 0.1171 - val_categorical_accuracy: 1.0000 - val_loss: 2.4173e-04
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 928ms/step - categorical_accuracy: 1.0000 - loss: 0.1046 - val_categorical_accuracy: 1.0000 - val_loss: 3.6030e-04
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 896ms/step - categorical_accuracy: 1.0000 - loss: 0.0792 - val_categorical_accuracy: 1.0000 - val_loss: 0.0061
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 2s 1s/step - categorical_accuracy: 0.9864 - loss: 0.083

In [71]:
model.save('action4wdCNN_F_Banglalink_Bn_F10.h5')

In [72]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 162ms/step


In [73]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [74]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[0, 0],
        [0, 1]]], dtype=int64)

In [75]:
accuracy_score(ytrue, yhat)

1.0

In [76]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [79]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model  # Assuming the CNN model is pre-trained and saved

# Initialize video capture
cap = cv2.VideoCapture(0)

# Load your pre-trained CNN model
model = load_model('F:/wordsign/action4wdCNN_F_Banglalink_Bn_F10.h5')  # Replace with the correct model path

# Mediapipe setup
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

# Define actions
actions = np.array(['Amar', 'Sonar', 'Bangla', 'Ami', 'Valobashi'])  # List of actions your model detects
threshold = 0.7  # Confidence threshold to trigger action detection
sequence = []
sentence = []

# Function to extract keypoints (same as before)
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, face, lh, rh])

# Start mediapipe holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        
        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 1. Extract keypoints
        keypoints = extract_keypoints(results)
        
        # 2. Update sequence with new keypoints
        sequence.append(keypoints)
        sequence = sequence[-30:]  # Keep only the last 30 frames
        
        if len(sequence) == 30:
            # 3. Make prediction using the CNN model
            sequence_array = np.expand_dims(sequence, axis=0)  # Add batch dimension
            sequence_array = sequence_array.reshape((1, 30, 1662, 1))  # Reshape for CNN input (batch_size, seq_len, keypoints, channels)
            res = model.predict(sequence_array)[0]  # Predict for the sequence
            
            # Get the action with the highest probability
            predicted_action = actions[np.argmax(res)]
            print(f'Predicted Action: {predicted_action} - Probability: {res[np.argmax(res)]}')
            
            # 4. Update the sentence if the confidence is above the threshold
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])
            
            # Limit the sentence to the last 5 actions
            if len(sentence) > 5:
                sentence = sentence[-5:]

        # 5. Display predicted action (sentence)
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (10,30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully on 'q' key
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 235ms/step
Predicted Action: Ami - Probability: 0.6820234656333923
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
Predicted Action: Ami - Probability: 0.9007206559181213
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
Predicted Action: Ami - Probability: 0.8159357905387878
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step
Predicted Action: Ami - Probability: 0.5777179598808289
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
Predicted Action: Amar - Probability: 0.7699354290962219
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step
Predicted Action: Bangla - Probability: 0.4282640218734741
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
Predicted Action: Bangla - Probability: 0.4941447377204895
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Predicted Action: Bangla - Probability: 0.7935094237327576
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 92ms/step
Predicted Action: Bangla - Probability: 0.39580607414245605
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Predicted Action: Ami - Probability: 0.6696402430534363
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predict